<a href="https://colab.research.google.com/github/AmirMoghadamFalahi/sample_task/blob/main/model/information_extraction_v0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/'My Drive'/autonews_data/

/content/drive/My Drive/autonews_data


In [ ]:
#library imports
import numpy as np
import pandas as pd
import os
import time
import json

In [ ]:
files = sorted(os.listdir())
files

['$search_results_50k_202010151034.csv',
 '$search_results_50k_202010151034_10.csv',
 '$search_results_50k_202010151034_11.csv',
 '$search_results_50k_202010151034_12.csv',
 '$search_results_50k_202010151034_13.csv',
 '$search_results_50k_202010151034_14.csv',
 '$search_results_50k_202010151034_15.csv',
 '$search_results_50k_202010151034_16.csv',
 '$search_results_50k_202010151034_17.csv',
 '$search_results_50k_202010151034_18.csv',
 '$search_results_50k_202010151034_19.csv',
 '$search_results_50k_202010151034_2.csv',
 '$search_results_50k_202010151034_20.csv',
 '$search_results_50k_202010151034_21.csv',
 '$search_results_50k_202010151034_22.csv',
 '$search_results_50k_202010151034_23.csv',
 '$search_results_50k_202010151034_24.csv',
 '$search_results_50k_202010151034_25.csv',
 '$search_results_50k_202010151034_26.csv',
 '$search_results_50k_202010151034_27.csv',
 '$search_results_50k_202010151034_28.csv',
 '$search_results_50k_202010151034_3.csv',
 '$search_results_50k_202010151034_4.

In [ ]:
t1 = time.time()

for i, file_name in enumerate(files):
  print(i, file_name)

  if i == 0:
    autonews_df = pd.read_csv(file_name)
    columns = autonews_df.columns
  else:
    df = pd.read_csv(file_name, names=columns)
    autonews_df = pd.concat([autonews_df, df])
    # print(autonews_df.head())
  
print('elapsed time:', str(round(time.time() - t1)))
autonews_df = autonews_df.reset_index(drop=True)
autonews_df = autonews_df.replace({np.nan: None})

0 $search_results_50k_202010151034.csv
1 $search_results_50k_202010151034_10.csv
2 $search_results_50k_202010151034_11.csv
3 $search_results_50k_202010151034_12.csv
4 $search_results_50k_202010151034_13.csv
5 $search_results_50k_202010151034_14.csv
6 $search_results_50k_202010151034_15.csv
7 $search_results_50k_202010151034_16.csv
8 $search_results_50k_202010151034_17.csv
9 $search_results_50k_202010151034_18.csv
10 $search_results_50k_202010151034_19.csv
11 $search_results_50k_202010151034_2.csv
12 $search_results_50k_202010151034_20.csv
13 $search_results_50k_202010151034_21.csv
14 $search_results_50k_202010151034_22.csv
15 $search_results_50k_202010151034_23.csv
16 $search_results_50k_202010151034_24.csv
17 $search_results_50k_202010151034_25.csv
18 $search_results_50k_202010151034_26.csv
19 $search_results_50k_202010151034_27.csv
20 $search_results_50k_202010151034_28.csv
21 $search_results_50k_202010151034_3.csv
22 $search_results_50k_202010151034_4.csv
23 $search_results_50k_2020

In [ ]:
autonews_df.shape

(50042, 9)

In [ ]:
autonews_df.head()

,id,article_datetime,article_timestamp,category,access_control,headline,link,got_single,paragraph_dic
0,1,2020-10-10,1.60233e+09,Marketing,Subscription Required,Carmaker Honda tilts scale to trucks,/marketing/carmaker-honda-tilts-scale-trucks,True,"{""1"": ""LOS ANGELES — A year ago, the mantra be..."
1,4,2020-10-09,1.60224e+09,Suppliers,Subscription Required,Yanfeng system attacks COVID-19 inside cars wi...,/suppliers/yanfeng-system-attacks-covid-19-ins...,True,"{""1"": ""Interior supplier Yanfeng has revealed ..."
2,6,2020-10-10,1.60235e+09,Dealers,Subscription Required,Phone calls take new priority in pandemic,/dealers/phone-calls-take-new-priority-pandemic,True,"{""1"": ""More customers are buying cars using a ..."
3,7,2020-10-10,1.6023e+09,Suppliers,Subscription Required,The new kink in automotive hiring: Amazon,/suppliers/new-kink-automotive-hiring-amazon,True,"{""1"": ""As if it hasn't been hard enough recrui..."
4,8,2020-10-10,1.60228e+09,Cars & Concepts,Subscription Required,"Nissan dumps vans in U.S. and Canada, eyes new...",/cars-concepts/nissan-dumps-vans-us-and-canada...,True,"{""1"": ""Nissan is shifting strategy on commerci..."


Comming back to the root

In [ ]:
cd ../../..

/content


# Preparing data

In [ ]:
def prepare_list_data(dataset):

  news_lst = []
  id_lst = []

  for i in range(dataset.shape[0]):
    dic = json.loads(dataset.paragraph_dic[i])
    whole_news = str(dataset.headline[i]) + '\n' + '\n'.join(list(dic.values()))
    news_lst.append(whole_news)

    id_lst.append(dataset.id[i])

  return news_lst, id_lst

In [ ]:
news_lst, id_lst = prepare_list_data(dataset=autonews_df)

In [ ]:
def prepare_data_haystack(dataset):

  final_dicts = []

  for i, dic in enumerate(dataset.paragraph_dic):
    dic = json.loads(dic, encoding='utf-8')
    txt = ('\n'.join(dic.values()))
    dic = {'text': txt, 'meta': {'category': dataset.category[i], 'headline': dataset.headline[i], 
                                'datetime': dataset.article_datetime[i], 'id': dataset.id[i]}}
    final_dicts.append(dic)

  return final_dicts

In [ ]:
data_haystack = prepare_data_haystack(dataset=autonews_df)

# Required works for using Haystack

Setting up the Haystack library

In [ ]:
! pip install farm-haystack

     |████████████████████████████████| 81kB 4.1MB/s 
     |████████████████████████████████| 143kB 21.8MB/s 
     |████████████████████████████████| 317kB 24.0MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 5.5MB 19.1MB/s 
     |████████████████████████████████| 81kB 10.3MB/s 
     |████████████████████████████████| 92kB 6.8MB/s 
     |████████████████████████████████| 7.2MB 12.0MB/s 
     |████████████████████████████████| 194kB 43.9MB/s 
     |████████████████████████████████| 225kB 44.1MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 983kB 41.7MB/s 
     |████████████████████████████████| 3.0MB 36.9MB/s 
     |████████████████████████████████| 61kB 8.8MB/s 
     |████████████████████████████████| 4.9MB 38.5MB/s 
     |████████████████████████████████| 40kB 6.0MB/s 
     |████████████████████████████████| 2.5MB 22.2MB/s 
     |████████████████████████████████| 47.7MB 61kB/s 
     |

Installing ElasticSearch and connecting to a local hosted db

In [ ]:
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.6.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.6.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.6.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.6.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

In [ ]:
# Connect to Elasticsearch

from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

10/15/2020 19:26:55 - INFO - elasticsearch -   PUT http://localhost:9200/document [status:200 request:0.458s]
10/15/2020 19:26:56 - INFO - elasticsearch -   PUT http://localhost:9200/label [status:200 request:0.247s]


Storing the data on the ElasticSearch

In [ ]:
len(document_store.get_all_documents())

10/15/2020 19:26:56 - INFO - elasticsearch -   POST http://localhost:9200/document/_search?scroll=5m&size=1000 [status:200 request:0.079s]
10/15/2020 19:26:56 - INFO - elasticsearch -   DELETE http://localhost:9200/_search/scroll [status:200 request:0.015s]


0

In [ ]:
document_store.write_documents(data_haystack)

10/15/2020 19:27:09 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.729s]
10/15/2020 19:27:10 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.136s]
10/15/2020 19:27:11 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.064s]
10/15/2020 19:27:12 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.027s]
10/15/2020 19:27:13 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.013s]
10/15/2020 19:27:14 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.997s]
10/15/2020 19:27:15 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.000s]
10/15/2020 19:27:17 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.058s]


In [ ]:
len(document_store.get_all_documents())

10/15/2020 19:28:58 - INFO - elasticsearch -   POST http://localhost:9200/document/_search?scroll=5m&size=1000 [status:200 request:0.323s]
10/15/2020 19:28:58 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll [status:200 request:0.133s]
10/15/2020 19:28:58 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll [status:200 request:0.145s]
10/15/2020 19:28:58 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll [status:200 request:0.096s]
10/15/2020 19:28:58 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll [status:200 request:0.089s]
10/15/2020 19:28:58 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll [status:200 request:0.102s]
10/15/2020 19:28:58 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll [status:200 request:0.092s]
10/15/2020 19:28:59 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll [status:200 request:0.094s]
10/15/2020 19:28:59 - INFO - elasticsearch

50042

## Creating Retreiver, Reader and Finder

In [ ]:
from haystack.retriever.sparse import ElasticsearchRetriever
from haystack.reader.farm import FARMReader
from haystack import Finder

In [ ]:
retriever = ElasticsearchRetriever(document_store=document_store)
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)
finder = Finder(reader, retriever)

10/15/2020 19:29:11 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
10/15/2020 19:29:11 - INFO - farm.infer -   Could not find `deepset/roberta-base-squad2` locally. Try to download from model hub ...
10/15/2020 19:29:11 - INFO - filelock -   Lock 140021778862936 acquired on /root/.cache/torch/transformers/f7d4b9379a9c487fa03ccf3d8e00058faa9d664cf01fc03409138246f48760da.c6288e0f84ec797ba5c525c923a5bbc479b47c761aded9734a5f6a473b044c8d.lock


10/15/2020 19:29:12 - INFO - filelock -   Lock 140021778862936 released on /root/.cache/torch/transformers/f7d4b9379a9c487fa03ccf3d8e00058faa9d664cf01fc03409138246f48760da.c6288e0f84ec797ba5c525c923a5bbc479b47c761aded9734a5f6a473b044c8d.lock
10/15/2020 19:29:12 - INFO - filelock -   Lock 140021778862936 acquired on /root/.cache/torch/transformers/8c0c8b6371111ac5fbc176aefcf9dbe129db7be654c569b8375dd3712fc4dc67.d045adc91e17ecdf7dc3eeff4c875df94bdf2eb749d72b3ae47ae93f8e85213c.lock


10/15/2020 19:29:22 - INFO - filelock -   Lock 140021778862936 released on /root/.cache/torch/transformers/8c0c8b6371111ac5fbc176aefcf9dbe129db7be654c569b8375dd3712fc4dc67.d045adc91e17ecdf7dc3eeff4c875df94bdf2eb749d72b3ae47ae93f8e85213c.lock


10/15/2020 19:29:28 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
10/15/2020 19:29:36 - WARNING - farm.modeling.prediction_head -   Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"loss_ignore_index": -1}
10/15/2020 19:29:49 - INFO - filelock -   Lock 140019498772800 acquired on /root/.cache/torch/transformers/1e3af82648d7190d959a9d76d727ef629b1ca51b3da6ad04039122453cb56307.6a4061e8fc00057d21d80413635a86fdcf55b6e7594ad9e25257d2f99a02f4be.lock


10/15/2020 19:29:50 - INFO - filelock -   Lock 140019498772800 released on /root/.cache/torch/transformers/1e3af82648d7190d959a9d76d727ef629b1ca51b3da6ad04039122453cb56307.6a4061e8fc00057d21d80413635a86fdcf55b6e7594ad9e25257d2f99a02f4be.lock


...

10/15/2020 19:29:52 - INFO - filelock -   Lock 140019498772800 acquired on /root/.cache/torch/transformers/2d9b03b59a8af464bf4238025a3cf0e5a340b9d0ba77400011e23c130b452510.16f949018cf247a2ea7465a74ca9a292212875e5fd72f969e0807011e7f192e4.lock


10/15/2020 19:29:52 - INFO - filelock -   Lock 140019498772800 released on /root/.cache/torch/transformers/2d9b03b59a8af464bf4238025a3cf0e5a340b9d0ba77400011e23c130b452510.16f949018cf247a2ea7465a74ca9a292212875e5fd72f969e0807011e7f192e4.lock


10/15/2020 19:29:53 - INFO - filelock -   Lock 140019498772800 acquired on /root/.cache/torch/transformers/507984f2e28c7dfed5db9a20acd68beb969c7f2833abc9e582e967fa0291f3dc.100c88dbe27dbd73822c575274ade4eb2427596ac56e96769249b7512341654d.lock


10/15/2020 19:29:53 - INFO - filelock -   Lock 140019498772800 released on /root/.cache/torch/transformers/507984f2e28c7dfed5db9a20acd68beb969c7f2833abc9e582e967fa0291f3dc.100c88dbe27dbd73822c575274ade4eb2427596ac56e96769249b7512341654d.lock


10/15/2020 19:29:54 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
10/15/2020 19:29:54 - INFO - farm.infer -   Got ya 1 parallel workers to do inference ...
10/15/2020 19:29:54 - INFO - farm.infer -    0 
10/15/2020 19:29:54 - INFO - farm.infer -   /w\
10/15/2020 19:29:54 - INFO - farm.infer -   /'\
10/15/2020 19:29:54 - INFO - farm.infer -   


In [ ]:
prediction = finder.get_answers(question="partner", top_k_retriever=10000, top_k_reader=500)

10/15/2020 19:29:56 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.586s]
10/15/2020 19:29:56 - INFO - haystack.retriever.sparse -   Got 3533 candidates from retriever
10/15/2020 19:29:56 - INFO - haystack.finder -   Reader is looking for detailed answer in 14201146 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  3.39 Batches/s]
10/15/2020 19:31:22 - ERROR - farm.modeling.predictions -   Both start and end offsets should be 0: 
2997, 2997 with a no_answer. 
Inferencing Samples: 100%|██████████| 2/2 [00:02<00:00,  1.32s/ Batches]


In [ ]:
prediction['answers'][10]

{'answer': 'Mitsubishi Motors',
 'context': " slow-selling EVs and the business of Nissan's new global partner, Mitsubishi Motors Corp.\nSaikawa said that the new Leaf EV arriving in Japan this ye",
 'document_id': '36926b8a-5e3f-4086-9952-d5d6e5aabb99',
 'meta': {'category': 'Executives',
  'datetime': '2017-04-10',
  'headline': 'New Nissan boss puts focus on 2 trouble spots',
  'id': 27692},
 'offset_end': 84,
 'offset_end_in_doc': 972,
 'offset_start': 67,
 'offset_start_in_doc': 955,
 'probability': 0.7697837623281527,
 'score': 9.65672492980957}

In [ ]:
id_array = np.array(id_lst)

print(news_lst[int(np.where(id_array == 27692)[0])])

New Nissan boss puts focus on 2 trouble spots
YOKOHAMA, Japan -- Nissan will re-spark its electric vehicle sales and Mitsubishi will play a bigger role in the U.S.
Those are two of the top orders of business for Hiroto Saikawa, the first Japanese CEO of Nissan Motor Co. in 16 years, as he steps into a role held since 2001 by the aggressive Carlos Ghosn.
Saikawa tells Automotive News in his first interview with overseas media since taking office on April 1 that he will continue building on the successes Ghosn brought to Japan's No. 2 automaker, most recently through the six-year Nissan Power 88 business plan.
"Part of the next plan is going to be steady growth built on the foundation we made in the last six years," Saikawa said last week at Nissan's global headquarters here south of Tokyo. "You can say it's a kind of remaining homework from Nissan Power 88."
But more immediately, two trouble spots need attention: Nissan's slow-selling EVs and the business of Nissan's new global partner,

In [ ]:
import spacy
from spacy import displacy
from collections import Counter

In [ ]:
! python -m spacy download en_core_web_md
import en_core_web_md

nlp = en_core_web_md.load()

     |████████████████████████████████| 96.4MB 1.2MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051305 sha256=f4efb308e124f7c1e70db4208e85ecc694307b40671ec4bc87036c5feff736be
  Stored in directory: /tmp/pip-ephem-wheel-cache-15hzwfk5/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [ ]:
partners = []

for pred in prediction['answers']:
  answer_text = pred['context']
  # id = prediction['answers'][10]['meta']['id']
  # text = news_lst[int(np.where(id_array == id)[0])]
  doc = nlp(answer_text)
  sentences = [x for x in doc.sents]
  ner_array = np.array([(X.text, X.label_) for X in doc.ents])
  try:
    companies = np.unique(ner_array[np.where(ner_array[:, 1] == 'ORG')[0], 0])
    if len(companies) >= 2:
      partners.append(companies)
  except:
    continue

In [ ]:
partners

[array(['Audi', 'Baidu', 'Ford'], dtype='<U12'),
 array(['Dongfeng', 'Nissan', 'Renault'], dtype='<U8'),
 array(['Nissan', 'Renault'], dtype='<U8'),
 array(['Ghosn', 'Renault', 'alliance', 'anagement'], dtype='<U19'),
 array(['BMW AG', 'Baidu'], dtype='<U6'),
 array(['Renault', 'Reuters', 'closi'], dtype='<U7'),
 array(['EVs', 'Mitsubishi Motors Corp.', 'Nissan'], dtype='<U23'),
 array(['Renault', 'alliance', 'mbattled'], dtype='<U8'),
 array(['Chery Automobile', 'JLR', 'XFL'], dtype='<U16'),
 array(['Nissan', 'hosn'], dtype='<U6'),
 array(['BMW AG', 'osts'], dtype='<U6'),
 array(['EyeSight', 'Hitachi'], dtype='<U8'),
 array(['Renault', 'alliance'], dtype='<U8'),
 array(['GM', 'Ventec Life Systems'], dtype='<U19'),
 array(['Juno', 'Volkswagen'], dtype='<U10'),
 array(['Yanfeng', 'Yanfeng Automotive Interiors'], dtype='<U28'),
 array(['FCA', 'Guangzhou Automobile Group', 'olding Group'], dtype='<U26'),
 array(['Chongqing Changan Automobile Co.', 'Ford'], dtype='<U32'),
 array(['DeNA Co.

In [ ]:
displacy.render(doc, jupyter=True, style='ent')